# How are in-demand skills trending for Data Analysts?

## Methodology

1. Aggregate skill counts monthly
2. Re-analyze based on percentage of total jobs
3. Plot the monthly skill demand

### Original Exploration:
[Exercise - Trending Skills](../2_RF_Advanced/2_11_Pandas_Applying_Explode_Exercise.ipynb)

In [1]:
# Importing Libraries
import ast
import pandas as pd
import seaborn as sns
from datasets import load_dataset
import matplotlib.pyplot as plt 

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

### Any Trends?

Now that we found out what are the top 5 skills sought after in data roles, we can now dive deeper into this. Specifically, we want to find out if there is any trend with these skills, like *should we be worried that any of these technologies are going out of style, or are they actually booming?*